In [1]:
import torch
# from diffusers import DiffusionPipeline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

d:\Anaconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# pipe = DiffusionPipeline.from_pretrained("Hyeon2/riffusion-musiccaps", torch_dtype=torch.float16).to(device)

# prompt = "a jazz solo, enegetic with syncopation"
# image = pipe(prompt).images[0]

d:\Anaconda\lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
100%|██████████| 50/50 [00:31<00:00,  1.56it/s]


In [47]:
from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_params import SpectrogramParams
from pydub import AudioSegment

params = SpectrogramParams(
    stereo = True,
    max_frequency=20000,
)
converter = SpectrogramImageConverter(params)

audio_path = r"D:\Documents\OctAI\Text2Music\downloaded_files\electro\2 Beers and a Good Send.mp3"
audio_segment = AudioSegment.from_mp3(audio_path)

#5110 ms generate image of 512x512
img = converter.spectrogram_image_from_audio(audio_segment[0:5110])

In [48]:
rev_audio = converter.audio_from_spectrogram_image(img)

In [49]:
rev_audio

In [50]:
img.size

(512, 512)

In [51]:
def chunk_audio_by_ms(audio_segment, ms):
    chunks = []
    for i in range(0, len(audio_segment), ms):
        chunks.append(audio_segment[i:i+ms])
    return chunks


In [53]:
chunks = chunk_audio_by_ms(audio_segment, 5110)

In [57]:
chunks[-1]

In [58]:
def audio_chunks_to_images(chunks, converter):
    images = []
    for chunk in chunks:
        images.append(converter.spectrogram_image_from_audio(chunk))
    return images

In [60]:
spectrograms = audio_chunks_to_images(chunks, converter)

In [94]:
import pandas as pd
import os
import numpy as np

In [174]:
def get_row_by_titles(df, titles):
    return df.loc[df["title"].isin(titles)].drop_duplicates()

def get_all_files_in_folder(path):
    return [f"{path}/{file}" for file in os.listdir(path)]

def get_title_from_path(path):
    return path.split("/")[-1].split(".")[0]

def string_list_to_list(string):
    # transform a string of list to an actual list in python
    replaced = string[1:-1].replace("'", "")
    if "," in replaced:
        return replaced.split(", ")
    else:
        return [replaced]

In [155]:
wav_path = r"D:\Documents\OctAI\Text2Music\downloaded_files\electro"
epi_csv = r"D:\Documents\OctAI\Text2Music\epidemic_music.csv"

df = pd.read_csv(epi_csv)

In [156]:
test_path = get_all_files_in_folder(wav_path)[0]
titles = [get_title_from_path(path) for path in get_all_files_in_folder(wav_path)]

In [175]:
filtered_df = get_row_by_titles(df, titles)

filtered_df.genres = filtered_df.genres.apply(string_list_to_list)
filtered_df.metadataTags = filtered_df.metadataTags.apply(string_list_to_list)
filtered_df.moods = filtered_df.moods.apply(string_list_to_list)

In [176]:
def generate_pseudo_label(df):
    return ",".join(df["genres"]) + "," + ",".join(df["metadataTags"]) + "," + ",".join(df["moods"])

In [177]:
filtered_df.apply(generate_pseudo_label, axis=1)

1864     electro,acoustic-group,glowing,sunshine,moving...
1993     electro,acoustic-group,melancholy,disappointme...
2155     electro,acoustic-group,autumn,rain,warm,analog...
5885     traditional-blues,electro,bouncy,timber,vintag...
5925     modern-blues,electro,lyrics,guitar,waves,smoot...
                               ...                        
28931           electro,drama,tears,grief,piano,sad,dreamy
29171    electro,strange-weird,electronic,change of key...
33335    alternative-hip-hop,usa,electronic,synth,lofi,...
43999    electro,synth-pop,sentimental,action,90s,shuff...
44085    electro,synth-pop,gym,pumping,distorted,game,d...
Length: 1163, dtype: object